### 기사 크롤링

- 기사 크롤링은 일반적인 크롤링과는 다르게 크롤러가 두 개 필요합니다.
- 기사 주소를 받아올 크롤러 하나와 기사 주소를 얻어왔을때 얻어온 주소를 순회하는 크롤러 하나가 필요합니다.
- 따라서 먼저 기사 주소를 가져오는 크롤러를 만들어보겠습니다.
- 링크주소는 통상적으로 a태그 내부의 href=""속성에 들어있습니다.
- 이 주소를 가져오려면 .text를 사용하지 말아야 합니다.

### 링크주소 크롤러 제작

- 날짜와 마지막 페이지를 입력하면 해당 날짜에 업로드된 야구 기사 주소를 긁어옵니다.
- 주소의 경우 모든 경로가 다 적혀있지 않기 때문에 이 점을 감안해서 저장해야 합니다.
- 네이버 기사의 경우 주소를 긁어온 링크대로 사용하면 이동이 안 됩니다.
    - 1. 주소 앞에 https://sports.news.naver.com 를 추가로 붙여줘야 하고
    - 2. 링크 주소 내의 ; 를 & 로 교체해줘야 합니다.

In [2]:
# 라이브러리 임포트
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import codecs

In [58]:
# 드라이버 가동
driver = webdriver.Chrome('../chromedriver.exe')
base = 'https://sports.news.naver.com/'

# 가져온 링크들을 저장할 수 있는 빈 리스트 작성
link_list = []

# https://sports.news.naver.com/kbaseball/news/index.nhn?date=20191123&isphoto=N&page=1
# 위 주소로 접근

idx = 1

for count in range(12):
    driver.get('https://sports.news.naver.com/kbaseball/news/index.nhn?date=20191123&isphoto=N&page=%d' %(count+1))
    time.sleep(3)

    # 전체 페이지 소스 가져오기
    source = driver.page_source

    # 파싱하기
    ps = BeautifulSoup(source, 'html.parser')

    # 파싱한 데이터에서 기사영역만 가져오기
    articles = ps.find_all('div', id='_newsList')

    # 기사 링크를 획득하기 위한 a태그만 가져오기
    a = articles[0].find_all('a', class_="title")
    for items in a :
        path = str(items).replace('amp;', '').split('="')[2].split('" ')[0]
        link_list.append(base+path)
        
driver.close()
print('작업 완료')

In [60]:
# 1. a태그의 주소 앞에  https://sports.news.naver.com/ 를 붙여주어야 함.
# 2. 주소값에서 amp;이 없어져야 함
# 3. 얻어온 태그는 string 자료형으로 캐스팅 해주어야 함

len(link_list)
link_list

['https://sports.news.naver.com//kbaseball/news/read.nhn?oid=396&aid=0000532651',
 'https://sports.news.naver.com//kbaseball/news/read.nhn?oid=057&aid=0001398175',
 'https://sports.news.naver.com//kbaseball/news/read.nhn?oid=055&aid=0000774251',
 'https://sports.news.naver.com//kbaseball/news/read.nhn?oid=445&aid=0000017017',
 'https://sports.news.naver.com//kbaseball/news/read.nhn?oid=109&aid=0004124874',
 'https://sports.news.naver.com//kbaseball/news/read.nhn?oid=076&aid=0003500760',
 'https://sports.news.naver.com//kbaseball/news/read.nhn?oid=016&aid=0001605746',
 'https://sports.news.naver.com//kbaseball/news/read.nhn?oid=472&aid=0000018493',
 'https://sports.news.naver.com//kbaseball/news/read.nhn?oid=417&aid=0000475114',
 'https://sports.news.naver.com//kbaseball/news/read.nhn?oid=082&aid=0000960273',
 'https://sports.news.naver.com//kbaseball/news/read.nhn?oid=018&aid=0004522981',
 'https://sports.news.naver.com//kbaseball/news/read.nhn?oid=277&aid=0004577537',
 'https://sports

### 기사 수집 크롤러

- link_list에 저장된 주소를 이용해서 각 기사별 페이지로 이동
- 이동한 페이지에 있는 기사 본문을 txt파일에 저장

In [91]:
# 크롬드라이버 세팅
driver = webdriver.Chrome('../chromedriver.exe')

# 전체 코드 가져오기
for idx in range(len(link_list)) :
    f = codecs.open('./news/%d.txt' %(idx+1), encoding='utf-8', mode='w')
    driver.get(link_list[idx])
    time.sleep(2)
    
    source = driver.page_source

    # 파싱하기
    ps = BeautifulSoup(source, "html.parser")

    # 본문기사 가져오기
    content = ps.find_all('div', class_="news_end font1 size3", id='newsEndContents')
    st = str(content[0]).replace('<br/>', '\r\n')
    st= BeautifulSoup(st, "html.parser")
    st = st.text.replace('\t', '').strip()
    f.write(st)
    f.close()
driver.close()
print('작업 완료')

작업 완료
